In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import plotly.graph_objects as go

data_path = Path("../data/crobex_history.csv")
df = pd.read_csv(data_path, sep=";", quotechar='"', decimal=",", parse_dates=["date"]).sort_values("date")
values = df["last_value"].dropna().values.reshape(-1, 1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)  # Skaliranje za stabilnije učenje RNN-a.

# Sliding window: LOOKBACK koraka ulaza → jedan izlaz (sljedeća vrijednost).
LOOKBACK = 20
X, y = [], []
for i in range(LOOKBACK, len(scaled)):
    X.append(scaled[i - LOOKBACK : i, 0])
    y.append(scaled[i, 0])
X, y = np.array(X), np.array(y)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
# RNN očekuje (samples, timesteps, features); ovdje jedna značajka po koraku.
X_train = X_train.reshape(-1, LOOKBACK, 1)
X_test = X_test.reshape(-1, LOOKBACK, 1)

# SimpleRNN: ulaz (LOOKBACK, 1), izlaz jedan skriveni vektor; Dense daje skalarnu predikciju.
model = Sequential([
    SimpleRNN(16, input_shape=(LOOKBACK, 1)),
    Dense(1)
])
model.compile(optimizer="adam", loss="mse")
model.fit(X_train, y_train, epochs=50, batch_size=128, verbose=0)


/Users/drazenbaric/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [ ]:
# Predikcije RNN-a su u skaliranom prostoru; inverse_transform vraća u originalne jedinice.
pred_train = scaler.inverse_transform(model.predict(X_train))
pred_test = scaler.inverse_transform(model.predict(X_test))
real_train = scaler.inverse_transform(y_train.reshape(-1, 1))
real = scaler.inverse_transform(y_test.reshape(-1, 1))

# metrics
mse_train = mean_squared_error(real_train, pred_train)
mse_test = mean_squared_error(real, pred_test)
mae_train = mean_absolute_error(real_train, pred_train)
mae_test = mean_absolute_error(real, pred_test)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

print("Train — MSE:", mse_train, " MAE:", mae_train, " RMSE:", rmse_train)
print("Test  — MSE:", mse_test, " MAE:", mae_test, " RMSE:", rmse_test)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step
Train — MSE: 119.36304950446196  MAE: 7.198321118871271  RMSE: 10.925339788970499
Test  — MSE: 51711.618564933044  MAE: 162.69689832694576  RMSE: 227.40188777785696


In [23]:
cutoff_idx = LOOKBACK + split
dates = df["date"].loc[df["last_value"].notna()].values
train_vals = scaler.inverse_transform(scaled[:cutoff_idx])

# plotly: train, real (test), predicted
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=dates[:cutoff_idx],
        y=train_vals.ravel(),
        name="train",
        mode="lines",
        line=dict(color="steelblue", width=1),
        opacity=0.8,
    )
)
fig.add_trace(
    go.Scatter(
        x=dates[cutoff_idx:],
        y=real.ravel(),
        name="real (test)",
        mode="lines",
        line=dict(color="steelblue", width=1.5),
    )
)
fig.add_trace(
    go.Scatter(
        x=dates[cutoff_idx:],
        y=pred_test.ravel(),
        name="predicted",
        mode="lines",
        line=dict(color="coral", width=1.5),
    )
)
fig.add_vline(x=dates[cutoff_idx], line_dash="dash", line_color="gray", line_width=1)
fig.update_layout(
    title="crobex – train, real, predicted",
    xaxis_title="date",
    yaxis_title="last_value",
    height=400,
    template="plotly_white",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)

fig.show()


In [22]:
# optional: train vs test metrics comparison (bar)
fig_metrics = go.Figure()
fig_metrics.add_trace(
    go.Bar(
        name="train",
        x=["MSE", "MAE", "RMSE"],
        y=[mse_train, mae_train, rmse_train],
        marker_color="steelblue",
    )
)
fig_metrics.add_trace(
    go.Bar(
        name="test",
        x=["MSE", "MAE", "RMSE"],
        y=[mse_test, mae_test, rmse_test],
        marker_color="coral",
    )
)
fig_metrics.update_layout(
    barmode="group",
    title="Train vs Test metrike",
    yaxis_title="vrijednost",
    template="plotly_white",
    height=350,
)
Path("../figures").mkdir(parents=True, exist_ok=True)
fig_metrics.write_image("../figures/predict-next-value-metrics.png")
fig_metrics.show()

ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
